In [1]:
#Potrebne biblioteke
import os
import shutil
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter

In [4]:
#Putanje do uzoraka u različitim kadrovima
path_HE = "C:/Users/Public/Uzorak_slika/HE"
path_Fine = "C:/Users/Public/Uzorak_slika/Fine"
path_Fokus = "C:/Users/Public/Uzorak_slika/Fokus"

### 1. Kreiranje stratuma (slojeva uzorka)

Cilj je uzorkovati podatke na takava način da oni slojevi koji su prisutni kod cijelovitog uzorka budu reflektirani u skupovima za učenje, validaciju i testiranje. Stratifikacija se radi prema **operateru** i **tipu proizvoda**, npr. udio uzorka u kojima operater **O1** radi na proizvodu **T1** treba biti sličan u sva tri podskupa cijelovitog uzorka. Ovaj pristup za cilj ima osigurati bolju generalizaciju modela.

In [5]:
#Lista uzoraka i list oznaka za strat
samples = sorted(os.listdir(path_HE), key=lambda x: int(x.split("_")[0])) #trik da budu sortirani prema broju uzorka
stratum_labels = [sample.split("_", maxsplit=3)[3] for sample in samples] #oznaka prema kojoj se radi stratifikacija npr. "O1_T1"
#len(samples), len(stratum_labels)
#len(np.unique(stratum_labels)) == 12 #Provjeri da li imaš 12 slojeva (4 operatera x 3 tipa proizvoda)

#Funkcija za izvlačenja udjela slojeva u uzorku
def stratum_counter(strat):
    '''Iz liste oznaka slojeva za svako opažanje u uzorku
    izračunava količinu i udio svakog sloja u uzorku
    
    Argumenti
    ---------
    strat : list
    Lista oznaka slojeva svakog opažanja unutar uzorka.
    
    Povratna vrijednost
    -------------------
    stratum_count: list
    Lista količina i udjela pojedinog sloja u uzorku
    
    '''
    counted_values = sorted(Counter(strat).items())
    total_samples = len(strat)
    stratum_count = [f'Sloj: {key} => count: {value} , perc: {(value/total_samples)*100:.2f}%' 
                 for key, value in counted_values]
    
    return stratum_count

#Koliko kojege sloja je prisutno u uzorku
sample_stratum_count = stratum_counter(stratum_labels)
sample_stratum_count

ValueError: invalid literal for int() with base 10: 'test'

Uzorak će biti podjeljen na slijedeći način:
* **480** opažanja u skupu za učenje
* **70** opažanja u skupu za validaciju
* **70** opažanja u skupu za testiranje

U ovom dijelu skripte radimo samo sa `Video_ID` oznakama uzoraka, kasnije ćemo podjeliti video zapise i njihove oznake.

In [5]:
#Podjela u skup za učenje_validaciju i testiranje
x_train_val, x_test = train_test_split(samples, test_size=70, random_state=42, stratify=stratum_labels)
len(x_train_val), len(x_test)

(550, 70)

In [6]:
#Ispitivanje udjela slojeva u skupu za učenje_validaciju i testiranje

#Oznake slojeva u train_val i test skupu
train_val_strata = [sample.split("_", maxsplit=3)[3] for sample in x_train_val]
test_strata = [sample.split("_", maxsplit=3)[3] for sample in x_test]

#Koliko kojeg sloja ima u train_val i test skupu
train_val_count = stratum_counter(train_val_strata)
test_count = stratum_counter(test_strata)

#Usporedba udjela pojedinog sloja
print("Oznaka:","Uzorak:", "Train_val:", "Test:")
print(len("Oznaka:") * "-", len("Uzorak:") * "-", len("Train_val:") * "-", len("Test:") * "-")
for label, kontrola, train, test in zip(np.unique(stratum_labels), sample_stratum_count, train_val_count, test_count):
    print(label, kontrola.split(":")[-1], train.split(":")[-1], test.split(":")[-1])

Oznaka: Uzorak: Train_val: Test:
------- ------- ---------- -----
O1_T1  8.06%  8.00%  8.57%
O1_T2  11.45%  11.45%  11.43%
O1_T3  10.81%  10.73%  11.43%
O2_T1  8.06%  8.00%  8.57%
O2_T2  7.90%  8.00%  7.14%
O2_T3  8.06%  8.00%  8.57%
O3_T1  7.74%  7.82%  7.14%
O3_T2  7.90%  8.00%  7.14%
O3_T3  7.26%  7.27%  7.14%
O4_T1  8.06%  8.00%  8.57%
O4_T2  7.74%  7.82%  7.14%
O4_T3  6.94%  6.91%  7.14%


In [8]:
#Podjela skupa za učenje na skup za učenje i validaciju
x_train, x_val = train_test_split(x_train_val, test_size=70, random_state=42, stratify=train_val_strata)
len(x_train), len(x_val)

(480, 70)

In [9]:
#Oznake slojeva u train i val skup
train_strata = [sample.split("_", maxsplit=3)[3] for sample in x_train]
val_strata = [sample.split("_", maxsplit=3)[3] for sample in x_val]

#Koliko kojeg sloja ima u train i val skupu
train_count = stratum_counter(train_strata)
val_count = stratum_counter(val_strata)

#Usporedba
print("Oznaka:","Uzorak:", "Train:", "Val:")
print(len("Oznaka:") * "-", len("Uzorak:") * "-", len("Train:") * "-", len("Val:") * "-")
for label, kontrola, train, val in zip(np.unique(stratum_labels), sample_stratum_count, train_count, val_count):
    print(label, kontrola.split(":")[-1], train.split(":")[-1], val.split(":")[-1])

Oznaka: Uzorak: Train: Val:
------- ------- ------ ----
O1_T1  8.06%  8.12%  7.14%
O1_T2  11.45%  11.46%  11.43%
O1_T3  10.81%  10.83%  10.00%
O2_T1  8.06%  7.92%  8.57%
O2_T2  7.90%  7.92%  8.57%
O2_T3  8.06%  7.92%  8.57%
O3_T1  7.74%  7.92%  7.14%
O3_T2  7.90%  7.92%  8.57%
O3_T3  7.26%  7.29%  7.14%
O4_T1  8.06%  7.92%  8.57%
O4_T2  7.74%  7.92%  7.14%
O4_T3  6.94%  6.88%  7.14%


### 2. Pregled opasnih opažanja

Ovo su vizualno totalno različita opažanja od uobičajenih stoga sada analiziram koliko je takvih u validacijskom i test skupu.

In [10]:
danger_data = np.array([287, 288, 328, 301, 304, 
                      308, 312, 313, 327, 513, 
                      518, 523, 525, 549, 517, 
                      571, 544, 557, 558, 570,
                      572, 573, 574, 575, 576,
                      577, 578, 579]) #28 ovakvih opažanja sva pripadaju O3 operateru

val_samples = [int(sample.split("_")[0]) for sample in x_val]
test_samples = [int(sample.split("_")[0]) for sample in x_test]

val_danger = [sample in val_samples for sample in danger_data]
test_danger = [sample in test_samples for sample in danger_data]

#Koja su to opažanja 
danger_data[val_danger], danger_data[test_danger]

(array([525, 558, 573, 576]), array([301, 513, 577]))

**Zaključak:** 4 složena opažanja u validacijskom setu, a 3 složena u skupu za testiranje. 

### 3. Kreiranje poddirektorija "train", "val" i "test" za "Uzorak_slika"

In [11]:
#Prebacivanje uzoraka iz direktorija "Uzorak_slika", u odgovarajuće poddirektorije "train", "val", "test"
#Polazne lokacije
slike_path = "C:/Users/Public/Uzorak_slika"
slike_subdirs = os.listdir(slike_path)

#Nazivi poddirektorija odredišnih lokacija
dest_loc = ["train", "val", "test"]
#Uzorci koje je potrebno rasporediti
data_splits = [x_train, x_val, x_test]

#Petlja po poddiretkorijima unutar datoteke "Uzorak_slika"
for subdir in slike_subdirs:
    #Istodobna petlja po nazivu odredišnog poddiretkorija i po listi koja kao podliste sadrži oznake videa podjeljenih u tri skupa
    for dest, split in zip(dest_loc, data_splits):
        #Kreiranje putanje do odredišnog direktorija
        dest_path = os.path.join(slike_path, subdir, dest) 
        if not os.path.exists(dest_path):
            os.mkdir(dest_path)
        #Petlja po polaznim datotekama i njihovo pomicanje na odredišni direktorij
        for path in split:
            src_path = os.path.join(slike_path, subdir, path)
            shutil.move(src_path, dest_path)
            #os.remove(src_path) Ovo netreba jer ne radi kopiju već pomak

Nakon što su opažanja za svaki kadar podjeljena u tri skupa, ovo isto je potrebno napraviti i za svaku oznaku.

In [12]:
#Razbijanje .txt sa oznakama na posebne fajlove za učenje, test, validaciju - OVO pokreni tek kad je gotovo označavanje!!!
label_names = ['train_labels', 'val_labels', 'test_labels']
data_splits = [x_train, x_val, x_test]

#Petlja po imenima datoteka oznaka i po nazivima opažanja u 3 skupa podataka
for label_name, data_split in zip(label_names, data_splits):
    #Otvori datoteku u koju češ zapisivati
    with open(f"C:/Users/Public/Labels/{label_name}.txt", "w") as labels:
        labels.write("Video_ID,Activity_ID,Activity_name,Start_time,End_time,Start_frame,End_frame\n")
        #Učitavaj jednu po jednu liniju (skalabilno rješenje, bolje nego odjednom učitati cijelu datoteku) 
        #iz datoteke iz koje prebacuješ - automatski će biti zatvoreno jer je isto u with kontekstu
        for line in open("C:/Users/Public/Labels/labels_all.txt", "r"):
            #Zapiši samo ona opažanja koja su u odgovarajućoj podjeli podataka (npr. samo one koji su u trening skupu)
            if line.split(",")[0] in data_split:
                labels.write(line)
                

In [13]:
#Još finija podjela - ako će biti potrebno, svaka oznaka opažanja u svoj fajl - Ovo pokreni tek kada je gotovo označavanje!!!
label_names = ['train_labels', 'val_labels', 'test_labels']
split_dirs = ['train', 'val', 'test']
data_splits = [x_train, x_val, x_test]

#Petlja po tri liste
for label_name, split_dir, data_split in zip(label_names, split_dirs, data_splits):
    #Za svako opažanje iz pojedine podjele podataka
    for sample in data_split:
        #Otvori datoteku za svako opažanje u koju češ zapisivati
        with open(f"C:/Users/Public/Labels/{split_dir}/{sample}.txt", "w") as labels:
            labels.write("Video_ID,Activity_ID,Activity_name,Start_time,End_time,Start_frame,End_frame\n")
            #Provjeri da li je opažanje u datoteci iz koje prebacuješ - više linija sadrži oznake istog opažanja
            for line in open(f"C:/Users/Public/Labels/{label_name}.txt", "r"):
                if line.split(",")[0] in sample:
                    labels.write(line)

In [9]:
##Kreiranje oznaka za svaku pojedinačnu sličicu

#Putanje do uzoraka u jednom od kadrova
samples_path = "C:/Users/Public/Uzorak_slika/HE"

#Labels path
labels_path = 'C:/Users/Public/Labels'

#Oznake poddirektorija iz kojih se kreiraju nove oznake
data_splits = ["train", "val", "test"]

#Nazivi poddirektorija odredišnih lokacija oznaka
dest_loc = ["train_all", "val_all", "test_all"]

#Funkcija za izvlačenje riječnika sa oznakom uzoraka te brojem slika u uzorku
def samp_size2dict(samples_path):
    '''Iz direktorija sa uzorcima izvlači broj slika u uzorku, pri čemu
    dokument mora biti formatiran prema naputcima.
    
    Argumenti
    ---------
    samples_path : Apsolutna putanja do direktorija sa uzorcima
    
    Povratna vrijednost
    -------------------
    size_dict: dict
    Riječnik koji za ključ ima oznaku uzorka, a za vrijednosti broj sličica
    
    '''
    size_dict = {sample: len(os.listdir(os.path.join(samples_path, sample))) for sample in os.listdir(samples_path)}
    
    return size_dict

#Funkcija za izvlačenje riječnika sa oznakom aktivnosti te početnim i završnim frejmom
def labs2dict(label_path):
    '''Iz csv dokumenta izvlači oznake aktivnosti, te početnu i 
    završnu sličicu aktivnosti, pri čemu dokument mora biti formatiran
    prema naputcima.
    
    Argumenti
    ---------
    label_path : Apsolutna putanja do csv dokumenta sa oznakama aktivnosti
    
    Povratna vrijednost
    -------------------
    lab_dict: dict
    Riječnik koji za ključ ima oznaku aktivnosti u obliku stringa, a za vrijednosti tuple(start_frame, end_frame)
    
    '''
    labels = pd.read_csv(label_path, header=0)
    lab_dict = {str(row[1]): (row[5], row[6]) for row in labels.values}
    
    return lab_dict

#Petlja po podjelama podataka i odredišnim lokacijama
for split, dest in zip(data_splits, dest_loc):
    #Putanja do oznaka u pojedinoj podjeli podataka
    labs_split_path = os.path.join(labels_path, split)
    #Izvlačenje broja slika po uzorcima u pojedinim podjelima podataka
    samp_split_path = os.path.join(samples_path, split)
    sample_sizes = samp_size2dict(samp_split_path)
    #Petlja po riječniku sa nazivima i veličinama uzorka
    for ime_uzorka, broj_slika in sample_sizes.items():
        #Izvlačenja riječnika sa oznakama aktivnosti i početnom i krajnjom sličicom
        labs_path = os.path.join(labs_split_path, f'{ime_uzorka}.txt')
        lab_dict = labs2dict(labs_path)
        #Za svaku sličicu promatranog uzorka daj oznaku s obzirom oznaku početka i kraja aktivnosti
        labels_list = []
        #Petlja po sličicam u uzroku
        for slika in range(1, broj_slika + 1):
            slika_oznaka = "0" #Oznaka pozadine je "0" 
            #Petlja po oznakama aktivnosti te oznakama početne i krajnje sličice aktivnosti
            for oznaka_act, raspon in lab_dict.items():
                #ako je broj sličice unutar raspona trajanja aktivnosti 
                #daj joj tu oznaku aktivnosti, inače ostaje 0 (oznaka pozadine)
                if raspon[0] <= slika <= raspon[1]:
                    slika_oznaka = oznaka_act
            #Dodaj oznaku promatranoj sličici unutar uzorka
            labels_list.append(slika_oznaka)
        #Zbog lakše učitavanja listu formatiram kao string
        labels_str = ",".join(labels_list)
        #Zapiši ovako označen uzorak (po svakoj sličici)
        with open(f'{os.path.join(labels_path, dest, ime_uzorka)}.txt', 'w') as label:
            label.write(f'{ime_uzorka},{labels_str}')
        

In [10]:
#Testiranje oznaka

#Putanja do trening oznaka po sličicama
frame_labs_path = 'C:/Users/Public/Labels/train_all'
labels_name= os.listdir(frame_labs_path)

labels = {}
for label in labels_name[:2]:
    with open(os.path.join(frame_labs_path, label)) as label:
        lab_elements = label.readline().split(",", maxsplit=1)
        video_id = lab_elements[0]
        frame_labs = lab_elements[1].split(",")
        frame_labs = [int(lab) for lab in frame_labs]
        labels[video_id] = frame_labs